In [1]:
# All required python standard libraries
import os
import time

In [2]:
# All torch related imports 
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms
from torch import nn, optim
import torchvision

In [3]:
# using cv2 to read an image
import cv2

In [4]:
# All sci-kit related imports 
import pandas as pd
import numpy as np

In [5]:
from tqdm.notebook import tqdm as tq

In [6]:
def accuracy_finder(predictions , labels):
    values, max_indices = torch.max(predictions, dim=1)
    #accuracy = ( max_indices == labels ).sum()/max_indices.size()[0]
    accuracy = ( max_indices == labels ).sum()
    return accuracy

In [7]:
def csv_preprocessor(base_dir:str, directory:str):
    return os.path.join(base_dir,directory).replace("\\","/")

In [8]:
def return_all_image_list_from_processed_csv(csv_file):
    ### This returns the entire list full of images to be loaded into cpu
    ###
    ###
    ALL_IMAGES = []
    start = time.time()
    for i, items in tq(enumerate(csv_file.iloc[:,1])):
        image = cv2.imread(items, cv2.COLOR_BGR2RGB)
        resized = cv2.resize(image,(224,224))
        ALL_IMAGES.append(resized)
    
    print("Tt took us approximately {} seconds".format(time.time()-start))  
    return ALL_IMAGES

In [9]:
# pytorch device configurations 
BATCH_SIZE = 8
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
train_csv_directory = os.path.join(os.getcwd(),'guides\\isolated-dataset-csv\\IsolatedTrain.csv')

In [11]:
class IsolatedCharacterDataset(Dataset):
    def __init__(self, csv_dir_path,  transforms=None, custom_transform=None ):
        ### complete dataset path
        self.dataset_csv = pd.read_csv(csv_dir_path, usecols=["labels","directory"])  
        self.dataset_csv["directory"] = self.dataset_csv["directory"].map(lambda x: csv_preprocessor(base_dir=str(os.getcwd()), directory=str(x)))
        self.dataset_csv_numpy = self.dataset_csv.to_numpy()
        self.ALL_IMAGES = return_all_image_list_from_processed_csv(csv_file=self.dataset_csv)
        
        ### labels
        self.labels = self.dataset_csv_numpy[:,0]
        ### images directories
        self.image_directories = self.dataset_csv_numpy[:,1]
        ### transformations to apply on images
        self.transforms = transforms
        
    def __getitem__(self, index):
        # convert labels to tensor 
        label = torch.tensor(self.labels[index])
        
        # load single image from list of all preloaded images
        image = self.ALL_IMAGES[index]
        
        if self.transforms:
            ## apply transforms 
            image = self.transforms(image)
            image = image.float()
        label = label.long().subtract(1)
        return image, label 
    
    def __len__(self):
        rows , _ = self.dataset_csv_numpy.shape
        return rows

In [12]:
DATA_NORMALIZER = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),transforms.Resize((224,224))])

In [13]:
TRAIN_DATASET = IsolatedCharacterDataset(csv_dir_path=train_csv_directory,transforms=DATA_NORMALIZER)


Tt took us approximately 9.617926120758057 seconds


In [14]:
TRAIN_LOADER = DataLoader(dataset=TRAIN_DATASET,batch_size=BATCH_SIZE,shuffle= True)

In [15]:
class block(nn.Module):
    def __init__(self, in_channels, intermediate_channels, identity_downsample=None, stride=1):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(intermediate_channels, intermediate_channels, kernel_size=3, stride=stride, padding=1,)
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(intermediate_channels, intermediate_channels * self.expansion, kernel_size=1, stride=1, padding=0,)
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride
    
    #Identity block
    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)
        #x = self.relu(x) #custom 

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

In [16]:
class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(block, layers[0], intermediate_channels=64, stride=1)
        self.layer2 = self._make_layer(block, layers[1], intermediate_channels=128, stride=2)
        self.layer3 = self._make_layer(block, layers[2], intermediate_channels=256, stride=2)
        self.layer4 = self._make_layer(block, layers[3], intermediate_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels change
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, intermediate_channels * 4, kernel_size=1, stride=stride,),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(block(self.in_channels, intermediate_channels, identity_downsample, stride))

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)

In [17]:
def ResNet50(img_channel=3, num_classes=171):
    return ResNet(block, [2, 3, 5, 2], img_channel, num_classes)
net = ResNet50()

In [18]:
optimizer =  optim.Adam(net.parameters(), lr=0.07) # learning rate 
# defining the loss function
criterion =  nn.CrossEntropyLoss()
net = net.to(DEVICE)
criterion = criterion.to(DEVICE)

In [19]:
from tqdm import tqdm 

In [22]:
def training(epochs:int):
    all_training_losses = []
    all_training_accuracy = []
    for epoch in tq(range(epochs)):
        total_epoch_loss = 0
        total_accuracy_epoch = 0
        for i, data in tqdm(enumerate(TRAIN_LOADER, 0)): 
            image,label = data
            optimizer.zero_grad(set_to_none=True)
            
            label = label.to(DEVICE)
            image = image.to(DEVICE)
            output = net(image)
            
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                print(output,label)
                total_epoch_loss += loss
                batches_training_accuracy = accuracy_finder(predictions=output, labels=label)
                total_accuracy_epoch = total_accuracy_epoch  + batches_training_accuracy   
            
            if i+1 % 1000 == 0: 
                print("Batch : {}/{}".format(i, len(TRAIN_LOADER))) 
        # total epoch loss 
        total_epoch_loss = total_epoch_loss / len(TRAIN_LOADER)
        # total epoch accuracy 
        total_accuracy_epoch = total_accuracy_epoch /len(TRAIN_LOADER)
        
        # display the epoch training loss
        print("epoch : {}/{}, loss = {:.8f}, acc = {:.8f}".format(epoch + 1, epochs, total_epoch_loss, total_accuracy_epoch ))
        all_training_losses.append(total_epoch_loss)
        all_training_accuracy.append(total_accuracy_epoch)
        
    print("Training completed")
    return all_training_accuracy, all_training_losses

In [ ]:
t_acc, t_loss = training(50)

1it [00:00,  2.10it/s]

tensor([[ -52.8453,  -52.0761,  -52.4239,  ...,  -52.3777,  -51.6944,
          -52.6524],
        [ -52.5069,  -51.7647,  -52.0854,  ...,  -52.0446,  -51.3818,
          -52.3134],
        [-225.8761, -227.9313, -228.3135,  ..., -228.8150, -228.7944,
         -227.8481],
        ...,
        [ -52.7307,  -51.9523,  -52.3015,  ...,  -52.2616,  -51.5829,
          -52.5240],
        [ -52.8812,  -52.1021,  -52.4496,  ...,  -52.4077,  -51.7139,
          -52.6738],
        [ -52.7279,  -51.9741,  -52.2941,  ...,  -52.2733,  -51.5909,
          -52.5402]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([141,  31,  72, 150,  85,  46,  54, 102], device='cuda:0')


2it [00:00,  2.08it/s]

tensor([[ -22.1193,  -22.0255,  -21.9458,  ...,  -21.8794,  -21.8682,
          -21.9354],
        [-242.6664, -242.5898, -243.8684,  ..., -243.2708, -242.7329,
         -243.8258],
        [ -21.0820,  -21.0541,  -20.9609,  ...,  -20.9039,  -20.9158,
          -20.9185],
        ...,
        [ -36.3211,  -36.1931,  -36.1892,  ...,  -36.0558,  -36.0604,
          -36.2113],
        [ -20.8528,  -20.8486,  -20.7446,  ...,  -20.6990,  -20.7206,
          -20.7015],
        [ -20.4120,  -20.4039,  -20.2905,  ...,  -20.2415,  -20.2600,
          -20.2631]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([ 28, 134, 109,  95, 143, 101,  95, 145], device='cuda:0')


3it [00:01,  2.06it/s]

...

tensor([[-0.6904, -0.3281, -0.4231,  ..., -1.1010, -0.2942, -0.3936],
        [-0.6963, -0.3277, -0.4178,  ..., -1.1009, -0.2940, -0.3942],
        [-0.6237, -0.2440, -0.4210,  ..., -1.0159, -0.2212, -0.2901],
        ...,
        [-0.7334, -0.4127, -0.4402,  ..., -1.1860, -0.3717, -0.4964],
        [-0.6237, -0.2440, -0.4210,  ..., -1.0159, -0.2212, -0.2901],
        [-0.6408, -0.2637, -0.4203,  ..., -1.0359, -0.2383, -0.3145]],
       device='cuda:0', grad_fn=<AddmmBackward>) tensor([ 53,   4, 162, 120, 127,  85, 162, 170], device='cuda:0')


124it [01:01,  2.00it/s]

tensor([[-0.6377, -0.2546, -0.3945,  ..., -1.0331, -0.2153, -0.2728],
        [-0.6656, -0.2930, -0.3937,  ..., -1.0717, -0.2494, -0.3178],
        [-0.7225, -0.4002, -0.4050,  ..., -1.1789, -0.3465, -0.4445],
        ...,
        [-0.7418, -0.3980, -0.3917,  ..., -1.1771, -0.3425, -0.4408],
        [-0.6355, -0.2520, -0.3947,  ..., -1.0305, -0.2130, -0.2697],
        [-0.6853, -0.3154, -0.3912,  ..., -1.0942, -0.2688, -0.3438]],
       device='cuda:0', grad_fn=<AddmmBackward>) tensor([139,  67, 155, 100,  22,  91, 142,  16], device='cuda:0')


125it [01:01,  2.01it/s]

tensor([[-0.6564, -0.2714, -0.3715,  ..., -1.0553, -0.2166, -0.2648],
        [-0.6560, -0.2709, -0.3715,  ..., -1.0549, -0.2162, -0.2643],
        [-0.7594, -0.4459, -0.3749,  ..., -1.2293, -0.3749, -0.4642],
        ...,
        [-0.6477, -0.2610, -0.3729,  ..., -1.0449, -0.2075, -0.2534],
        [-0.6477, -0.2610, -0.3729,  ..., -1.0449, -0.2075, -0.2534],
        [-0.6477, -0.2610, -0.3729,  ..., -1.0449, -0.2075, -0.2534]],
       device='cuda:0', grad_fn=<AddmmBackward>) tensor([ 67,  14,  43,  68, 137, 140, 141,  91], device='cuda:0')


In [35]:
DEVICE

device(type='cuda')